# AI Agents in Life Sciences: Hands-on Session 2 - AI agent collaboration with the Model Context Protocol (MCP)

Welcome! This notebook provides a complete, hands-on guide to MCP with clear theory explanations followed by practical implementations.

**Learning path:**
1.  **Core concepts:** MCP architecture and components
2.  **Basic primitives:** Resources and Tools
3.  **Advanced features:** Prompts, Streaming, and real-time interactions
4.   **Full implementation:** Complete MCP server and client

--- 
## Part 1: Core components & architecture

### 1.1 - The three components

MCP has three main roles that work together:

1. **MCP Host:** The AI application itself (like a chatbot)
2. **MCP Client:** Connects to servers on behalf of the Host  
3. **MCP Server:** Provides context, data, and tools



**Data flow:**
Host → Client → Server → Client → Host

**Example: MCP Host** 

```python
# MCP HOST (conceptual - uses the client)
def demo_basic_mcp():
    # Host uses client to interact with server
    client = MCPClient("http://localhost:8501/mcp")
    
    # Host decides what to request
    init_result = client.initialize()
    tools = client.list_tools()
    result = client.call_tool("get_protein_function", {"protein_id": "P53_HUMAN"})
    
    return result['result']['content'][0]['text']
```

**Example: MCP Client**

```python
# MCP CLIENT
class MCPClient:
    def __init__(self, server_url):
        self.server_url = server_url
        self.request_id = 1
        
    def _send_request(self, method, params=None):
        # Client handles JSON-RPC protocol
        payload = {
            "jsonrpc": "2.0",
            "id": self.request_id,
            "method": method
        }
        if params:
            payload["params"] = params
            
        self.request_id += 1
        # Transport: HTTP POST
        response = requests.post(self.server_url, json=payload)
        return response.json()
```

**Example: MCP Server**

```python
# MCP SERVER (Data & Tools Provider)
@app.route('/mcp', methods=['POST'])
def handle_mcp():
    # Server handles all MCP methods
    data = request.json
    
    # Validate JSON-RPC 2.0
    if data.get('jsonrpc') != '2.0':
        return jsonify({
            "jsonrpc": "2.0",
            "error": {"code": -32600, "message": "Invalid Request"},
            "id": None
        })
    
    method = data.get('method')
    params = data.get('params', {})
    request_id = data.get('id')
    
    # Handle different MCP methods
    if method == 'initialize':
        return handle_initialize(request_id)
    elif method == 'resources/list':
        return handle_resources_list(request_id)
    elif method == 'tools/call':
        return handle_tools_call(params, request_id)
```

### 1.2 - The Two-layer architecture

MCP is intentionally simple:

- **Data layer:** JSON-RPC 2.0 message format
- **Transport layer:** HTTP(S), STDIO, or other transports

**Separation of concerns:**
- Data layer defines WHAT we communicate
- Transport layer defines HOW we communicate

**Example: Data layer**

```python
# DATA LAYER: JSON-RPC 2.0 Structure

# Request Structure (Client -> Server)
request_payload = {
    "jsonrpc": "2.0",           # Protocol version
    "id": 1,                    # Request identifier  
    "method": "tools/call",     # What operation
    "params": {                 # Operation parameters
        "name": "get_protein_function",
        "arguments": {"protein_id": "P53_HUMAN"}
    }
}

# Response Structure (Server -> Client) 
response_payload = {
    "jsonrpc": "2.0",
    "id": 1,                    # Matches request ID
    "result": {                 # Successful result
        "content": [{
            "type": "text",
            "text": "Function of P53_HUMAN: Acts as a tumor suppressor."
        }]
    }
}

# Error Structure (Server -> Client)
error_payload = {
    "jsonrpc": "2.0",
    "id": 1,
    "error": {
        "code": -32602,         # Standard error code
        "message": "Protein not found"
    }
}
```

**Example: Transport layer**

```python
# TRANSPORT LAYER: HTTP Implementation

# Client Transport (basic_client.py)
response = requests.post(self.server_url, json=payload)

# Server Transport (basic_server.py)
@app.route('/mcp', methods=['POST'])
def handle_mcp():
    # Flask handles HTTP transport
    data = request.json          # Extract data layer
    # Process MCP method...
    return jsonify(response_payload)  # Send back via HTTP
```

**Example: Initialization Handshake**

```python
# MCP INITIALIZATION SEQUENCE

# Client sends initialize (basic_client.py)
def initialize(self):
    return self._send_request("initialize")

# Server responds (basic_server.py)  
if method == 'initialize':
    return jsonify({
        "jsonrpc": "2.0",
        "result": {
            "protocolVersion": "2024-11-05",
            "capabilities": {
                "resources": {},
                "tools": {},
                "logging": {}
            },
            "serverInfo": {
                "name": "Protein MCP Server",
                "version": "1.0.0"
            }
        },
        "id": request_id
    })

# Client acknowledges (basic_server.py)
elif method == 'notifications/initialized':
    return jsonify({
        "jsonrpc": "2.0",
        "result": None,
        "id": request_id
    })

```

**Example: Complete Flow**

```python
# COMPLETE MCP FLOW (from basic_client.py demo_basic_mcp)

def demo_basic_mcp():
    # 1. Host creates client
    client = MCPClient("http://localhost:8501/mcp")
    
    print("=== MCP Basic Client Demo ===\n")
    
    # 2. Initialize connection
    print("1. Initializing MCP connection...")
    init_result = client.initialize()
    
    # 3. Discover resources
    print("2. Listing available resources...")
    resources = client.list_resources()
    
    # 4. Read resource content  
    print("3. Reading protein resource...")
    protein_data = client.read_resource("protein://proteins")
    
    # 5. Discover tools
    print("4. Listing available tools...")
    tools = client.list_tools()
    
    # 6. Execute tool
    print("5. Calling tool to get protein function...")
    tool_result = client.call_tool("get_protein_function", {"protein_id": "P53_HUMAN"})
    
    # 7. Use result
    print(f"   Result: {tool_result['result']['content'][0]['text']}")
```

--- 
## Part 2: Resources and tools

### 2.1 - Understanding MCP resources

**Resources** are read-only data sources that servers expose:
- Databases or data collections
- File systems or document stores  
- API endpoints with static data

**Key methods:**
- `resources/list` - Discover available resources
- `resources/read` - Read resource content

**Example: Resource**

```python
# RESOURCE: List available resources
elif method == 'resources/list':
    return jsonify({
        "jsonrpc": "2.0",
        "result": {
            "resources": [{
                "uri": "protein://proteins",
                "name": "Protein Database",
                "description": "Sample protein data",
                "mimeType": "application/json"
            }]
        },
        "id": request_id
    })

# RESOURCE: Read resource content  
elif method == 'resources/read':
    uri = params.get('uri')
    if uri == 'protein://proteins':
        return jsonify({
            "jsonrpc": "2.0",
            "result": {
                "contents": [{
                    "uri": uri,
                    "mimeType": "application/json",
                    "value": json.dumps(protein_db)
                }]
            },
            "id": request_id
        })
```

### 2.2 - Understanding MCP tools

**Tools** are executable functions that perform actions:
- Calculations or data processing
- External API calls
- Database operations
- Any side-effect producing function

**Key methods:**
- `tools/list` - Discover available tools
- `tools/call` - Execute a tool with parameters

**Tool schema:** Each tool defines its input parameters with JSON Schema

**Example: Tool**


```python
# TOOL: List available tools
elif method == 'tools/list':
    return jsonify({
        "jsonrpc": "2.0",
        "result": {
            "tools": [{
                "name": "get_protein_function",
                "description": "Get protein function by ID",
                "inputSchema": {
                    "type": "object",
                    "properties": {
                        "protein_id": {
                            "type": "string", 
                            "description": "Protein ID (e.g., P53_HUMAN)"
                        }
                    },
                    "required": ["protein_id"]
                }
            }]
        },
        "id": request_id
    })

# TOOL: Execute a tool
elif method == 'tools/call':
    tool_name = params.get('name')
    arguments = params.get('arguments', {})
    
    if tool_name == 'get_protein_function':
        protein_id = arguments.get('protein_id')
        if protein_id in protein_db:
            return jsonify({
                "jsonrpc": "2.0",
                "result": {
                    "content": [{
                        "type": "text",
                        "text": f"Function: {protein_db[protein_id]['function']}"
                    }]
                },
                "id": request_id
            })
```

--- 
## Part 3: Combining everything and testing it all together

Now it’s time to try our first implementation of the MCP protocol. We’ll start by combining all the concepts we’ve covered into two separate files: ```basic_client.py``` and ```basic_server.py```.
As you go through the code, try to locate each of the sections we’ve discussed, This will help you understand the overall flow of the implementation.

### 3.1 - Create data, server and client scripts

We'll start with a simple JSON file to act as our protein database. **Run the cell below** to create `protein_db.json`.

In [3]:
%%writefile protein_db.json
{
  "P53_HUMAN": {"name": "Cellular tumor antigen p53", "organism": "Homo sapiens", "function": "Acts as a tumor suppressor.", "log": "Initializing analysis...\nSequence loaded...\nChecking for known mutation sites...\nSite R248Q found...\nGenerating report...\nAnalysis complete."},
  "P53_MOUSE": {"name": "Cellular tumor antigen p53", "organism": "Mus musculus", "function": "Key regulator of cell cycle and apoptosis.", "log": "Initializing analysis...\nSequence loaded...\nNo known mutation sites found...\nAnalysis complete."},
  "P0DTC2": {"name": "Spike glycoprotein", "organism": "SARS-CoV-2", "function": "Mediates entry into host cells.", "log": "Initializing analysis...\nSequence loaded...\nChecking for known mutation sites...\nSite D614G found...\nGenerating report...\nAnalysis complete."}
}

Writing protein_db.json


Now run the next section to generate the ```basic_server.py``` file in the same directory.

In [1]:
%%writefile basic_server.py
from flask import Flask, jsonify, request
import json

app = Flask(__name__)

with open('MCP_scratch/protein_db.json') as f:
    protein_db = json.load(f)

@app.route('/mcp', methods=['POST'])
def handle_mcp():
    data = request.json
    
    # Validate JSON-RPC 2.0
    if data.get('jsonrpc') != '2.0':
        return jsonify({
            "jsonrpc": "2.0",
            "error": {"code": -32600, "message": "Invalid Request"},
            "id": None
        })
    
    method = data.get('method')
    params = data.get('params', {})
    request_id = data.get('id')
    
    # Initialize method (MCP handshake)
    if method == 'initialize':
        return jsonify({
            "jsonrpc": "2.0",
            "result": {
                "protocolVersion": "2024-11-05",
                "capabilities": {
                    "resources": {},
                    "tools": {},
                    "logging": {}
                },
                "serverInfo": {
                    "name": "Protein MCP Server",
                    "version": "1.0.0"
                }
            },
            "id": request_id
        })
    
    # Resources methods
    elif method == 'resources/list':
        return jsonify({
            "jsonrpc": "2.0",
            "result": {
                "resources": [{
                    "uri": "protein://proteins",
                    "name": "Protein Database",
                    "description": "Sample protein data",
                    "mimeType": "application/json"
                }]
            },
            "id": request_id
        })
    
    elif method == 'resources/read':
        uri = params.get('uri')
        if uri == 'protein://proteins':
            return jsonify({
                "jsonrpc": "2.0",
                "result": {
                    "contents": [{
                        "uri": uri,
                        "mimeType": "application/json",
                        "value": json.dumps(protein_db)
                    }]
                },
                "id": request_id
            })
        return jsonify({
            "jsonrpc": "2.0",
            "error": {"code": -32601, "message": "Resource not found"},
            "id": request_id
        })
    
    # Tools methods
    elif method == 'tools/list':
        return jsonify({
            "jsonrpc": "2.0",
            "result": {
                "tools": [{
                    "name": "get_protein_function",
                    "description": "Get protein function by ID",
                    "inputSchema": {
                        "type": "object",
                        "properties": {
                            "protein_id": {"type": "string", "description": "Protein ID (e.g., P53_HUMAN)"}
                        },
                        "required": ["protein_id"]
                    }
                }]
            },
            "id": request_id
        })
    
    elif method == 'tools/call':
        tool_name = params.get('name')
        arguments = params.get('arguments', {})
        
        if tool_name == 'get_protein_function':
            protein_id = arguments.get('protein_id')
            if protein_id in protein_db:
                return jsonify({
                    "jsonrpc": "2.0",
                    "result": {
                        "content": [{
                            "type": "text",
                            "text": f"Function of {protein_id}: {protein_db[protein_id]['function']}"
                        }]
                    },
                    "id": request_id
                })
            return jsonify({
                "jsonrpc": "2.0",
                "error": {"code": -32602, "message": f"Protein {protein_id} not found"},
                "id": request_id
            })
        
        return jsonify({
            "jsonrpc": "2.0",
            "error": {"code": -32601, "message": "Tool not found"},
            "id": request_id
        })
    
    elif method == 'notifications/initialized':
        # Acknowledge initialization
        return jsonify({
            "jsonrpc": "2.0",
            "result": None,
            "id": request_id
        })
    
    return jsonify({
        "jsonrpc": "2.0",
        "error": {"code": -32601, "message": "Method not found"},
        "id": request_id
    })

if __name__ == '__main__':
    print("MCP Basic Server running on http://localhost:8501")
    print("This server follows the official MCP specification with JSON-RPC 2.0")
    app.run(port=8501)

Writing basic_server.py


Now run the next section to generate the ```basic_client.py``` file in the same directory.

In [2]:
%%writefile basic_client.py
import requests
import json

class MCPClient:
    def __init__(self, server_url):
        self.server_url = server_url
        self.request_id = 1
        
    def _send_request(self, method, params=None):
        payload = {
            "jsonrpc": "2.0",
            "id": self.request_id,
            "method": method
        }
        if params:
            payload["params"] = params
            
        self.request_id += 1
        response = requests.post(self.server_url, json=payload)
        return response.json()
    
    def initialize(self):
        return self._send_request("initialize")
    
    def list_resources(self):
        return self._send_request("resources/list")
    
    def read_resource(self, uri):
        return self._send_request("resources/read", {"uri": uri})
    
    def list_tools(self):
        return self._send_request("tools/list")
    
    def call_tool(self, name, arguments):
        return self._send_request("tools/call", {
            "name": name,
            "arguments": arguments
        })

# Demo the client
def demo_basic_mcp():
    client = MCPClient("http://localhost:8501/mcp")
    
    print("=== MCP Basic Client Demo ===\n")
    
    # 1. Initialize
    print("1. Initializing MCP connection...")
    init_result = client.initialize()
    print(f"   Server: {init_result['result']['serverInfo']['name']}")
    print(f"   Protocol: {init_result['result']['protocolVersion']}\n")
    
    # 2. List Resources
    print("2. Listing available resources...")
    resources = client.list_resources()
    for resource in resources['result']['resources']:
        print(f"   {resource['name']}: {resource['uri']}")
    print()
    
    # 3. Read Resource
    print("3. Reading protein resource...")
    protein_data = client.read_resource("protein://proteins")
    data = json.loads(protein_data['result']['contents'][0]['value'])
    print(f"   Found {len(data)} proteins in database\n")
    
    # 4. List Tools
    print("4. Listing available tools...")
    tools = client.list_tools()
    for tool in tools['result']['tools']:
        print(f"   {tool['name']}: {tool['description']}")
    print()
    
    # 5. Call Tool
    print("5. Calling tool to get protein function...")
    tool_result = client.call_tool("get_protein_function", {"protein_id": "P53_HUMAN"})
    print(f"   Result: {tool_result['result']['content'][0]['text']}")

if __name__ == '__main__':
    try:
        demo_basic_mcp()
    except requests.exceptions.ConnectionError:
        print("ERROR: Could not connect to MCP server. Is basic_server.py running?")

Writing basic_client.py


### 3.2 - Run the Server and Client

1.  Ensure that all required dependencies are installed correctly. For detailed setup instructions, see ```SciLifeLab_agent_workshop/README.md```

2.  Open a terminal and navigate to the project directory:

    ```bash
    cd Section_2_MCP/
    ```

3.  Activate the virtual environment if it is deactivated (see step 3.2.1 for details):

    ```bash
    source ../.venv/bin/activate
    ```

4.  Provide the correct API key that was sent to you via email in the ```MCP_scratch/.env``` file.
Replace ```"PASTE-YOUR_KEY-HERE"``` with your actual key, making sure to keep the double quotes.

    ```bash
    OPENAI_API_KEY="PASTE-YOUR_KEY-HERE"
    ```

5.  Start the server in the same terminal:

    ```bash
    python MCP_scratch/basic_server.py
    ```

    Keep this terminal open.  **Do not disconnect it.**

6.  Open another terminal and run the client:

    ```bash
    python MCP_scratch/basic_client.py
    ```

7.  Observe the output in both terminals to see the communication flow.


--- 
## Part 4: Advanced concepts

Basic primitives are for simple request-response. The true power of MCP comes from dynamic, two-way interactions.

### 4.1 - Understanding MCP prompts

**Prompts** are reusable conversation templates:
- Pre-defined message patterns
- Parameterizable templates
- Structured LLM interactions

**Key methods:**
- `prompts/list` - Discover available prompts
- `prompts/get` - Get a prompt template with arguments

**Example: MCP Prompt**

```python
# PROMPT: List available prompts
elif method == 'prompts/list':
    return jsonify({
        "jsonrpc": "2.0",
        "result": {
            "prompts": [{
                "name": "protein_analysis",
                "description": "Generate comprehensive protein analysis",
                "arguments": [{
                    "name": "protein_id",
                    "description": "ID of the protein to analyze",
                    "required": True
                }]
            }]
        },
        "id": request_id
    })

# PROMPT: Get prompt template
elif method == 'prompts/get':
    prompt_name = params.get('name')
    prompt_args = params.get('arguments', {})
    
    if prompt_name == 'protein_analysis':
        protein_id = prompt_args.get('protein_id')
        if protein_id in protein_db:
            protein = protein_db[protein_id]
            return jsonify({
                "jsonrpc": "2.0",
                "result": {
                    "description": f"Analysis of {protein['name']}",
                    "messages": [{
                        "role": "user",
                        "content": {
                            "type": "text",
                            "text": f"Analyze this protein:\\nName: {protein['name']}\\nOrganism: {protein['organism']}\\nFunction: {protein['function']}"
                        }
                    }]
                },
                "id": request_id
            })
```

### 4.2 - Advanced MCP features

**Notifications:** Server-push mechanism for real-time updates
- Server can proactively send information
- Great for data changes, alerts, progress updates

**Streaming:** Real-time data flow
- HTTP chunked transfer encoding
- Perfect for long-running processes

**Elicitation:** Asking for clarification
- Server can request more specific input
- Useful for ambiguous requests

**Sampling:** Server requests LLM generation
- Server provides prompt, client's LLM generates
- Enables collaborative AI tasks

--- 
## Part 5: Combining advanced features and testing it all together

Just like before, it’s time to implement the advanced features we discussed earlier.
We’ll begin by combining all the concepts into two separate files: ```advanced_client.py``` and ```advanced_server.py```.

As you explore the code, try to identify each of the sections we’ve covered. This will help you better understand the overall flow of the implementation.

### 5.1 - Create server and client scripts

Run the next section to generate the ```advanced_server.py``` file in the same directory.

In [4]:
%%writefile advanced_server.py
from flask import Flask, jsonify, request, Response, stream_with_context
import json
import uuid
import time
from threading import Thread
import logging

# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

app = Flask(__name__)

# Load data
with open('MCP_scratch/protein_db.json') as f:
    protein_db = json.load(f)

# Server state
server_state = {
    "clients": {},
    "pending_requests": {}
}

@app.route('/mcp', methods=['POST'])
def handle_mcp():
    data = request.json
    
    # Validate JSON-RPC 2.0
    if data.get('jsonrpc') != '2.0':
        return jsonify({
            "jsonrpc": "2.0",
            "error": {"code": -32600, "message": "Invalid Request"},
            "id": None
        })
    
    method = data.get('method')
    params = data.get('params', {})
    request_id = data.get('id')
    
    logger.info(f"MCP Method: {method}")
    
    # Initialize method (MCP handshake)
    if method == 'initialize':
        return jsonify({
            "jsonrpc": "2.0",
            "result": {
                "protocolVersion": "2024-11-05",
                "capabilities": {
                    "resources": {"listChanged": True},
                    "tools": {},
                    "logging": {},
                    "prompts": {}
                },
                "serverInfo": {
                    "name": "Advanced Protein MCP Server",
                    "version": "1.0.0"
                }
            },
            "id": request_id
        })
    
    # Resources methods
    elif method == 'resources/list':
        return jsonify({
            "jsonrpc": "2.0",
            "result": {
                "resources": [{
                    "uri": "protein://proteins",
                    "name": "Protein Database",
                    "description": "Sample protein data with streaming capability",
                    "mimeType": "application/json"
                }]
            },
            "id": request_id
        })
    
    elif method == 'resources/read':
        uri = params.get('uri')
        if uri == 'protein://proteins':
            return jsonify({
                "jsonrpc": "2.0",
                "result": {
                    "contents": [{
                        "uri": uri,
                        "mimeType": "application/json", 
                        "value": json.dumps(protein_db)
                    }]
                },
                "id": request_id
            })
        return jsonify({
            "jsonrpc": "2.0", 
            "error": {"code": -32601, "message": "Resource not found"},
            "id": request_id
        })
    
    # Tools methods with advanced features
    elif method == 'tools/list':
        return jsonify({
            "jsonrpc": "2.0",
            "result": {
                "tools": [
                    {
                        "name": "find_protein",
                        "description": "Find protein by name with disambiguation",
                        "inputSchema": {
                            "type": "object",
                            "properties": {
                                "protein_name": {"type": "string", "description": "Protein name to search for"}
                            },
                            "required": ["protein_name"]
                        }
                    },
                    {
                        "name": "analyze_protein_stream",
                        "description": "Stream protein analysis in real-time", 
                        "inputSchema": {
                            "type": "object",
                            "properties": {
                                "protein_id": {"type": "string", "description": "Protein ID to analyze"}
                            },
                            "required": ["protein_id"]
                        }
                    },
                    {
                        "name": "get_protein_hypothesis",
                        "description": "Generate research hypothesis for a protein",
                        "inputSchema": {
                            "type": "object", 
                            "properties": {
                                "protein_id": {"type": "string", "description": "Protein ID"}
                            },
                            "required": ["protein_id"]
                        }
                    }
                ]
            },
            "id": request_id
        })
    
    elif method == 'tools/call':
        tool_name = params.get('name')
        arguments = params.get('arguments', {})
        
        # Tool 1: Find protein with disambiguation (Elicitation simulation)
        if tool_name == 'find_protein':
            protein_name = arguments.get('protein_name', '').lower()
            
            # Simulate elicitation by returning multiple options
            if protein_name == 'p53':
                return jsonify({
                    "jsonrpc": "2.0",
                    "result": {
                        "content": [{
                            "type": "text",
                            "text": "Multiple proteins match 'p53'. Please specify:\n- P53_HUMAN (Human p53)\n- P53_MOUSE (Mouse p53)"
                        }],
                        "isError": False
                    },
                    "id": request_id
                })
            
            # Search for exact matches
            matches = []
            for protein_id, info in protein_db.items():
                if protein_name in info['name'].lower():
                    matches.append(f"{protein_id}: {info['name']} ({info['organism']})")
            
            if matches:
                return jsonify({
                    "jsonrpc": "2.0",
                    "result": {
                        "content": [{
                            "type": "text", 
                            "text": f"Found {len(matches)} proteins:\n" + "\n".join(f"- {m}" for m in matches)
                        }]
                    },
                    "id": request_id
                })
            return jsonify({
                "jsonrpc": "2.0",
                "error": {"code": -32602, "message": "No proteins found"},
                "id": request_id
            })
        
        # Tool 2: Streaming analysis
        elif tool_name == 'analyze_protein_stream':
            protein_id = arguments.get('protein_id')
            
            if protein_id not in protein_db:
                return jsonify({
                    "jsonrpc": "2.0",
                    "error": {"code": -32602, "message": "Protein not found"},
                    "id": request_id
                })
            
            # For streaming, we'd normally use Server-Sent Events or websockets
            # For this demo, we'll simulate streaming with delayed chunks
            def generate_stream():
                protein = protein_db[protein_id]
                log_lines = protein.get('log', '').split('\n')
                
                for i, line in enumerate(log_lines):
                    if line.strip():
                        chunk = {
                            "jsonrpc": "2.0",
                            "method": "tools/call/progress",
                            "params": {
                                "progress": {
                                    "progress": (i + 1) / len(log_lines),
                                    "message": line
                                }
                            }
                        }
                        # In real MCP, this would be a server-initiated notification
                        # For HTTP, we simulate with multiple responses
                        yield f"data: {json.dumps(chunk)}\n\n"
                        time.sleep(0.5)
            
            return Response(stream_with_context(generate_stream()), mimetype='text/plain')
        
        # Tool 3: Hypothesis generation (Sampling simulation)
        elif tool_name == 'get_protein_hypothesis':
            protein_id = arguments.get('protein_id')
            
            if protein_id not in protein_db:
                return jsonify({
                    "jsonrpc": "2.0", 
                    "error": {"code": -32602, "message": "Protein not found"},
                    "id": request_id
                })
            
            protein_info = protein_db[protein_id]
            
            # Return a prompt that the client can use with their LLM
            return jsonify({
                "jsonrpc": "2.0",
                "result": {
                    "content": [{
                        "type": "text",
                        "text": f"PROMPT FOR LLM: Based on this protein information, generate a novel research hypothesis.\n\nProtein: {protein_info['name']}\nOrganism: {protein_info['organism']}\nFunction: {protein_info['function']}\n\nHypothesis:"
                    }]
                },
                "id": request_id
            })
        
        return jsonify({
            "jsonrpc": "2.0",
            "error": {"code": -32601, "message": "Tool not found"},
            "id": request_id
        })
    
    # Prompts method (MCP standard for template-based generation)
    elif method == 'prompts/list':
        return jsonify({
            "jsonrpc": "2.0",
            "result": {
                "prompts": [{
                    "name": "protein_analysis",
                    "description": "Generate comprehensive protein analysis",
                    "arguments": [{
                        "name": "protein_id",
                        "description": "ID of the protein to analyze",
                        "required": True
                    }]
                }]
            },
            "id": request_id
        })
    
    elif method == 'prompts/get':
        prompt_name = params.get('name')
        prompt_args = params.get('arguments', {})
        
        if prompt_name == 'protein_analysis':
            protein_id = prompt_args.get('protein_id')
            if protein_id in protein_db:
                protein = protein_db[protein_id]
                return jsonify({
                    "jsonrpc": "2.0",
                    "result": {
                        "description": f"Analysis of {protein['name']}",
                        "messages": [{
                            "role": "user",
                            "content": {
                                "type": "text",
                                "text": f"Analyze this protein and provide insights:\n\nName: {protein['name']}\nOrganism: {protein['organism']}\nFunction: {protein['function']}"
                            }
                        }]
                    },
                    "id": request_id
                })
        
        return jsonify({
            "jsonrpc": "2.0",
            "error": {"code": -32601, "message": "Prompt not found"},
            "id": request_id
        })
    
    elif method == 'notifications/initialized':
        # Client is ready - we could send notifications here
        client_info = params.get('clientInfo', {})
        logger.info(f"Client initialized: {client_info}")
        return jsonify({
            "jsonrpc": "2.0",
            "result": None,
            "id": request_id
        })
    
    return jsonify({
        "jsonrpc": "2.0", 
        "error": {"code": -32601, "message": "Method not found"},
        "id": request_id
    })

# Background thread for simulating notifications
def send_periodic_notifications():
    """Simulate server-initiated notifications"""
    while True:
        time.sleep(30)  # Every 30 seconds
        # In real MCP with bidirectional transport, we'd send notifications here
        logger.info("Simulated: Server would send notification to clients now")

if __name__ == '__main__':
    print("MCP Advanced Server running on http://localhost:8502")
    
    # Start notification thread
    notification_thread = Thread(target=send_periodic_notifications, daemon=True)
    notification_thread.start()
    
    app.run(port=8502, debug=False)

Writing advanced_server.py


Run the next section to generate the ```advanced_client.py``` file in the same directory.

In [5]:
%%writefile advanced_client.py
import requests
import json
import time
from openai import OpenAI
from dotenv import load_dotenv
import os

load_dotenv()

class MCPAdvancedClient:
    def __init__(self, server_url):
        self.server_url = server_url
        self.request_id = 1
        self.llm_client = OpenAI() if os.getenv('OPENAI_API_KEY') else None
        
    def _send_request(self, method, params=None):
        payload = {
            "jsonrpc": "2.0",
            "id": self.request_id,
            "method": method
        }
        if params:
            payload["params"] = params
            
        self.request_id += 1
        response = requests.post(self.server_url, json=payload)
        return response.json()
    
    def initialize(self):
        return self._send_request("initialize")
    
    def list_tools(self):
        return self._send_request("tools/list")
    
    def call_tool(self, name, arguments):
        return self._send_request("tools/call", {
            "name": name,
            "arguments": arguments
        })
    
    def list_prompts(self):
        return self._send_request("prompts/list")
    
    def get_prompt(self, name, arguments=None):
        params = {"name": name}
        if arguments:
            params["arguments"] = arguments
        return self._send_request("prompts/get", params)

def demo_advanced_mcp():
    client = MCPAdvancedClient("http://localhost:8502/mcp")
    
    print("=== MCP Advanced Client Demo ===\n")
    
    # 1. Initialize
    print("1. Initializing advanced MCP connection...")
    init_result = client.initialize()
    print(f"Connected to {init_result['result']['serverInfo']['name']}\n")
    
    # 2. List and demonstrate tools
    print("2. Available advanced tools:")
    tools = client.list_tools()
    for tool in tools['result']['tools']:
        print(f"{tool['name']}: {tool['description']}")
    print()
    
    # 3. Demonstrate elicitation-like behavior
    print("3. Testing protein search (elicitation simulation)...")
    search_result = client.call_tool("find_protein", {"protein_name": "p53"})
    print(f"   {search_result['result']['content'][0]['text']}\n")
    
    # 4. Demonstrate hypothesis generation
    print("4. Testing hypothesis generation (sampling simulation)...")
    hypothesis_result = client.call_tool("get_protein_hypothesis", {"protein_id": "P0DTC2"})
    prompt_text = hypothesis_result['result']['content'][0]['text']
    print(f"   Prompt: {prompt_text.split('PROMPT FOR LLM: ')[1]}...")
    
    # Use LLM if available
    if client.llm_client:
        try:
            completion = client.llm_client.chat.completions.create(
                model="gpt-3.5-turbo",
                messages=[{"role": "user", "content": prompt_text}]
            )
            hypothesis = completion.choices[0].message.content
            print(f"LLM Hypothesis: {hypothesis}...\n")
        except Exception as e:
            print(f"LLM call failed: {e}\n")
    else:
        print("Set OPENAI_API_KEY to see LLM sampling in action\n")
    
    # 5. Demonstrate prompts feature
    print("5. Testing MCP prompts feature...")
    prompts = client.list_prompts()
    for prompt in prompts['result']['prompts']:
        print(f"{prompt['name']}: {prompt['description']}")
    
    prompt_result = client.get_prompt("protein_analysis", {"protein_id": "P53_HUMAN"})
    print(f"Prompt messages: {len(prompt_result['result']['messages'])} message(s)\n")
    
    print("6. Streaming simulation...")
    print("(In real MCP with bidirectional transport, streaming would happen here)")
    print("This demo shows the progress notification pattern\n")

if __name__ == '__main__':
    try:
        demo_advanced_mcp()
    except requests.exceptions.ConnectionError:
        print("ERROR: Could not connect to MCP server. Is advanced_server.py running?")
    except Exception as e:
        print(f"ERROR: {e}")

Writing advanced_client.py


### 5.2 - Run the Server and Client

1.  Ensure that all required dependencies are installed correctly. For detailed setup instructions, see ```SciLifeLab_agent_workshop/README.md```

2.  Open a terminal and navigate to the project directory:

    ```bash
    cd Section_2_MCP/
    ```

3.  Activate the virtual environment if it is deactivated (see step 5.2.1 for details):

    ```bash
    source ../.venv/bin/activate
    ```

4.  Provide the correct API key that was sent to you via email in the ```MCP_scratch/.env``` file.
Replace ```"PASTE-YOUR_KEY-HERE"``` with your actual key, making sure to keep the double quotes.

    ```bash
    OPENAI_API_KEY="PASTE-YOUR_KEY-HERE"
    ```

5.  Start the server in the same terminal:

    ```bash
    python MCP_scratch/advanced_server.py
    ```

    Keep this terminal open.  **Do not disconnect it.**

6.  Open another terminal and run the client:

    ```bash
    python MCP_scratch/advanced_client.py
    ```

7.  Observe the output in both terminals to see the communication flow.

--- 
## Part 6: Official MCP SDK implementation

### Why We built from scratch first

In this workshop, we intentionally implemented MCP using raw REST APIs and Flask to help you understand:

**What's Happening Under the Hood**
- JSON-RPC 2.0 protocol mechanics
- HTTP transport layer details  
- Raw request/response structures
- Error handling patterns

**Core Concepts First**
- Resources, Tools, Prompts as fundamental building blocks
- Transport vs Data layer separation
- Client-Server communication patterns

#### The Official SDK approach

While our custom implementation is great for learning, **production applications should use the official MCP SDK**:

**Benefits of the Official SDK:**
- **Protocol Compliance**: Handles JSON-RPC 2.0 correctly out of the box
- **Type Safety**: Full Python type hints and validation
- **Error Handling**: Built-in standardized error responses
- **Transport Abstraction**: Support for HTTP, SSE, and STDIO
- **Tooling Integration**: Works seamlessly with AI assistants

#### Ready-to-use SDK implementation

We've already implemented the same protein database functionality using the **official MCP Python SDK**!

**Location**: `Section_2_MCP/MCP_python_SDK/`

**What's Included:**
- `mcp_basic_server.py` - Basic resources and tools
- `mcp_basic_client.py` - Basic client using SDK
- `mcp_advanced_server.py` - Advanced features (prompts, streaming, sampling)
- `mcp_advanced_client.py` - Advanced client with OpenAI integration
- `protein_db.json` - Enhanced protein database

#### Next steps

1. **Go through** the `Section_2_MCP/MCP_python_SDK/README.md` file to get a clear understanding.
1. **Explore the SDK implementation** to see how much cleaner and more maintainable it is
2. **Compare the patterns** between our custom implementation and the SDK version
3. **Use the SDK for your own projects** - it's production-ready!

The SDK implementation demonstrates the same concepts but with proper abstractions, error handling, and best practices.

---

## Part 7: Bonus / optional step — connect to an existing host (Postman)

In this part, you’ll learn how to connect your running MCP servers to a real-world host application. We’ll use Postman, which includes built-in MCP support, to act as our client. This will allow you to interactively explore your server’s tools and resources—just as an AI assistant would.

### Next step

**Go through** the `Section_2_MCP/existing_clients_and_servers/README.pdf` file and follow the instructions provided there.

## Workshop complete!

---

**Congratulations!**

You’ve now explored MCP from its core principles all the way to production-ready implementations and integrations with existing applications. You understand:

**The protocol**: JSON-RPC 2.0 over HTTP  
**The architecture**: Host-Client-Server model  
**The primitives**: Resources, Tools, Prompts  
**Advanced features**: Streaming, Sampling, Notifications  
**Implementation paths**: Both custom and SDK-based approaches